# 03: Session Management & Context 🧠

This notebook demonstrates the **Session Management** layer of the SalesOps Agent Suite.

### 🎯 Goals
1.  **Initialize Session Service:** Set up the `SalesSessionService` (In-Memory) to manage conversation history.
2.  **Connect to Runner:** Integrate the session service with the Google ADK `Runner`.
3.  **Verify Persistence:** Execute a multi-turn conversation to ensure the agent "remembers" context (e.g., user name) across different interactions.

### 🏗️ Components Used
* `memory.session_service`: Custom wrapper for ADK's session management.
* `google.adk.runners.Runner`: Orchestrates the agent execution loop.
* [cite_start]`google.adk.sessions.InMemorySessionService`: Stores active conversation state[cite: 1802, 1803].

## Imports

In [ ]:
# Environment & Imports
import sys
import os
import asyncio
from dotenv import load_dotenv

# 1. Load Environment Variables (API Key)
# This must happen BEFORE we initialize Gemini
load_dotenv()

# 2. Add project root to path
# We go up one level from 'notebooks' to reach the root
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

# 3. Imports
from google.adk.runners import Runner
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.genai import types

# Import our session service
from memory.session_service import SalesSessionService

print(f"✅ Libraries loaded. Project root: {project_root}")
# Optional: Check if key is loaded (prints True/False, doesn't reveal key)
print(f"✅ API Key loaded: {'GOOGLE_API_KEY' in os.environ}")

c:\Users\YASH\anaconda3\envs\salesops\Lib\site-packages\google\cloud\aiplatform\models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


✅ Libraries loaded. Project root: d:\01. Github\salesops-suite
✅ API Key loaded: True


## 1: Initialize Service

In [2]:
# Initialize our custom wrapper
session_wrapper = SalesSessionService()
session_service = session_wrapper.get_service()

print("✅ Session Service Instantiated.")

Session Service initialized (In-Memory)
✅ Session Service Instantiated.


## 2: Setup Test Agent

In [3]:
# Cell 3: Create a Smarter Test Agent
# We give it explicit instructions to use conversation history
agent = LlmAgent(
    name="session_test_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction=(
        "You are a helpful SalesOps assistant. "
        "You maintain context across the conversation. "
        "If a user tells you their name or details, remember it and use it in future responses."
    ),
)

# Initialize the Runner with our Session Service
runner = Runner(agent=agent, app_name="salesops_demo", session_service=session_service)
print("✅ Runner Initialized with Smart Session Agent")

✅ Runner Initialized with Smart Session Agent


## 3: Run Chat (Test Persistence)

In [ ]:
async def run_chat(user_text, session_id):
    print(f"\n--- Session: {session_id} ---")
    print(f"User: {user_text}")

    # 1. Try to GET existing session
    session = await session_service.get_session(
        app_name="salesops_demo", user_id="user_1", session_id=session_id
    )

    # 2. If not found (None), CREATE it
    if session is None:
        print("(Creating new session...)")
        session = await session_service.create_session(
            app_name="salesops_demo", user_id="user_1", session_id=session_id
        )

    # 3. Run Agent
    query = types.Content(role="user", parts=[types.Part(text=user_text)])
    async for event in runner.run_async(
        user_id="user_1", session_id=session.id, new_message=query
    ):
        if event.is_final_response() and event.content:
            print(f"Agent: {event.content.parts[0].text}")


# Run two turns.
await run_chat("Hi, my name is Yash.", "session_001")
await run_chat("What is my name?", "session_001")


--- Session: session_001 ---
User: Hi, my name is Yash.
(Creating new session...)
Agent: Hi Yash, it's nice to meet you! How can I help you today?

--- Session: session_001 ---
User: What is my name?
Agent: Your name is Yash.


## ⏭️ Next Step: Building the Anomaly Agent

Great work! We have a working **Session Service** that can persist conversation history.

Now that our infrastructure (Ingestion, KPIs, Sessions) is ready, we move to **Day 4: Statistical Anomaly Detection**.

**Next, we will:**
1. Build the `AnomalyStatAgent` to mathematically detect sales spikes and drops.
2. Use the features we engineered to calculate Z-Scores.
3. Feed these anomalies into our AI agent to explain *why* they happened.

👉 **Proceed to `notebooks/04_feature_pipeline.ipynb` (if you haven't run it) or `notebooks/05_anomaly_detection.ipynb`.**